This notebook produces maps for each of the reaches compared. generate_report_data.ipynb needs to be run before this will work

In [ ]:
import dotenv
import os
from dotenv import load_dotenv
load_dotenv()
import logging

logging.getLogger().setLevel('WARNING')

output_folder = os.environ['OUTPUT_FOLDER']
output_folder


In [ ]:
from hydrological_connectivity.definitions.definitions_generator_factory import DefinitionsGeneratorFactory
from hydrological_connectivity.processing.model_result_aggregator import ModelResultAggregator
import logging
from hydrological_connectivity.definitions.model_definitions import DepthModelType, ModelDefinitions
from hydrological_connectivity.postprocessing.model_benchmark_cards import ModelBenchmarkCards
logging.getLogger().setLevel('WARNING')


model_types = ModelDefinitions.get_model_definitions()
# display(model_types)

exclude_1956 = ["1956"]
exclude_non_1956 = ['LBS - Culgoa FP South',
                    'LBS - Culgoa FP North',
                    'LBS - Narran River',
                    'LBS - St George',
                    'Namoi - DS Mollee',
                    'Namoi - Boolcarroll',
                    'Namoi - DS Gunidgera',
                    'Namoi - Merah North', '2013', '2016']

# exclude_list = exclude_non_1956
exclude_list = exclude_1956
definition = DefinitionsGeneratorFactory.get_generator(exclude_list)



In [ ]:
definition.generate()


In [ ]:

definition.simple_outputs[0]

In [ ]:
import pickle

output = open(
    f'{output_folder}{os.path.sep}product_results.pkl', 'rb')
product_results = pickle.load(output)
output.close()
output = open(
    f'{output_folder}{os.path.sep}overall_results.pkl', 'rb')
overall_results = pickle.load(output)
output.close()
output = open(
    f'{output_folder}{os.path.sep}overall_metrics.pkl', 'rb')
overall_metrics = pickle.load(output)
output.close()


In [ ]:
p_selected = {k: v for k, v in product_results.items() if 
              (k.depth_model_type == DepthModelType.FwDET) or
              (k.depth_model_type == DepthModelType.TVD and k.depth_model_params['seperation_of_waterbodies'] == 'ind')
    or ('accumulation_threshold' in k.depth_model_params and type(k.depth_model_params['accumulation_threshold'])==dict and 'best MAE' in k.depth_model_params['accumulation_threshold'])}
scene_list = list(list(p_selected.values())[0].keys())


In [ ]:
scene_id = scene_list[0]


In [ ]:
selected_model_scene = {model:scene_values[scene_id] for (model,scene_values) in p_selected.items()}

In [ ]:
aggregator_lookup= {}
aggregator_outputs = {}
last_outputs=[]
for model in p_selected.keys():
    model_result_aggregator = ModelResultAggregator(
        definition, model)
    aggregator_lookup[model] = model_result_aggregator
    aggregator_outputs[model] = model_result_aggregator.identified_outputs
    last_outputs = model_result_aggregator.identified_outputs




In [ ]:

def set_map_font_size(self, small=8, medium=10, large=12):
    SMALL_SIZE = small
    MEDIUM_SIZE = medium
    BIGGER_SIZE = large

    pyplot.style.use('seaborn-white')

    # controls default text sizes
    pyplot.rc('font', size=SMALL_SIZE)
    # fontsize of the axes title
    pyplot.rc('axes', titlesize=SMALL_SIZE)
    # fontsize of the x and y labels
    pyplot.rc('axes', labelsize=MEDIUM_SIZE)
    # fontsize of the tick labels
    pyplot.rc('xtick', labelsize=SMALL_SIZE)
    # fontsize of the tick labels
    pyplot.rc('ytick', labelsize=SMALL_SIZE)
    pyplot.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    # fontsize of the figure title
    pyplot.rc('figure', titlesize=BIGGER_SIZE)

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib import colorbar
from matplotlib import colors, cm
import seaborn
import numpy
from matplotlib import pyplot, ticker
from rasterio.plot import show
import os.path
import logging

for output_number in range(len(last_outputs)):
    print(output_number)

    model_results = {}
    for model in p_selected.keys():
        model_result_aggregator = ModelResultAggregator(
            definition, model)
        model_result_aggregator.identified_outputs = model_result_aggregator.identified_outputs[
            output_number:output_number+1]
        task = model_result_aggregator.produce_stats()
        model_results[model] = task[0]

    seaborn.set_theme(style="whitegrid")
    cmap = pyplot.get_cmap('RdYlBu')

    aim_width = (8.3-1)
    aim_height = task[0]["stats"].comparison.shape[0] / \
        task[0]["stats"].comparison.shape[1]*aim_width

    fig, axes_array = pyplot.subplots(
        nrows=2, ncols=2, figsize=(aim_width, aim_height), dpi=300, gridspec_kw={'left': 0, 'right': 1, 'top': 1, 'bottom': 0, 'wspace': 0, 'hspace': 0}, constrained_layout=False, frameon=False)

    axes_list = [axes_array[0][0], axes_array[1]
                [0], axes_array[0][1], axes_array[1][1]]

    model_results_list = list(model_results.items())
    model_results_list.reverse()
    subcaption = ['a) ', 'b) ', 'c) ']

    for map_index in range(3):
        ax = axes_list[map_index]
        (model, result) = model_results_list[map_index]
        show(result["stats"].comparison, transform=result["stats"].comp_transform, interpolation='nearest',
            cmap=cmap, vmin=-2, vmax=2, ax=ax)
        ax.set_title(subcaption[map_index] +
                    str(model.depth_model_type.name), y=0.85)
        ax.tick_params(axis='y', direction='in', pad=-10)
        ax.tick_params(axis='x', direction='in', pad=-10)
        for pos in ['top', 'bottom', 'right', 'left']:
            ax.spines[pos].set_edgecolor('black')
            ax.spines[pos].set_linewidth(0.5)
        ax.grid(linewidth=0.5)
        ax.xaxis.set_major_formatter(
            ticker.FuncFormatter(lambda tick_val, tick_pos: f'{tick_val // 10000:.0f}{tick_val % 10000:04.0f}'))
        ax.yaxis.set_major_formatter(
            ticker.FuncFormatter(lambda tick_val, tick_pos: f'{tick_val // 10000:.0f}{tick_val % 10000:04.0f}'))
        ax.get_xaxis().set_major_locator(ticker.MaxNLocator(5))
        ax.get_yaxis().set_major_locator(ticker.MaxNLocator(4))
        ax.tick_params(axis='both', which='major', labelsize=4)
        for tick in ax.get_yticklabels():
            tick.set_rotation(90)
            tick.set_va('center')

    ax = axes_list[3]
    ax.axis('off')
    # width="7%", height="85%", loc=1)
    cbaxes = ax.inset_axes([0.05, 0.85, 0.9, 0.07])
    cb = fig.colorbar(cm.ScalarMappable(norm=colors.Normalize(
        vmin=-2, vmax=2), cmap=cmap), cax=cbaxes, orientation='horizontal', label='Truth minus predicted')

    cbaxes.xaxis.set_label_position = 'bottom'
    cbaxes.xaxis.set_ticks_position = 'bottom'

    ax.text(0.1, 0.15, str(result["input"].short_description), ha='left')
    #pyplot.tight_layout(pad=0)
    #pyplot.show()
    pyplot.savefig(output_folder +
                   f"{os.path.sep}Reach_Comparison_{output_number:02d}")
    pyplot.cla()
    pyplot.clf()
    pyplot.close('all')
    pyplot.close(fig)
